In [ ]:

5 Cross-validation
As a last step in the tutorial we will study a cross-validation (CV) example.
The first step is to define 10 CV groups:
> Ind.cv <- createCV(mesa.model, groups=10, min.dist=.1)
> Ind.cv[1:10]
[1] 1 3 8 2 10 4 3 10 10 4
Here Ind.cv is a 4577 vector defining the CV-groups. Each element of the
vector indicates for which of the 10 CV-groups that the corresponding observation
should be left out. For the i
th group, we are going to use our
30
5. Cross-validation
2000 2002 2004 2006 2008 2010
3.0 4.0
60370002
time
2000 2002 2004 2006 2008 2010
3.0 4.0 5.0
60371601
time
2000 2002 2004 2006 2008 2010
2.0 3.5 5.0
60590001
time
2000 2002 2004 2006 2008 2010
3.5 4.5 5.5
L002
Figure 4: The predicted and observed data for 4 of the 25 locations. The
red-lines denote observations, the black line and grey shading give predictions
and 95% confidence intervals at unobserved time-points. The green and blue
give the contribution to the predictions from the regression and regression +
β-fields respectively.
model to predict at the observations marked by the number of that group
(i.e. Ind.cv = i), using all other observations (i.e. Ind.cv 6= i). Once we have
done this for each CV-group we can compare our predictions to the truth
and calculate cross-validated statistics such as RMSE and R2
.
However first we will take a closer look at the CV-groupings.
> table(Ind.cv)
Ind.cv
1 2 3 4 5 6 7 8 9 10
31
Tutorial for SpatioTemporal
438 389 811 556 546 165 228 487 160 797
We see that the number of observations left out of each group is rather
uneven; the main goal of createCV is to create CV-groups such that the
groups contain roughly the same number of locations ignoring the number of
observations at each location. If there are large differences in the number
of observations at different locations one could use the subset option to
create different CV-groupings for different types of locations. If the groups
are computed using a logical matrix (option Icv.vector=FALSE) instead of
a vector, it is possible to combine the resulting CV-groups. As an example,
group locations with more than 270 observations separately.
> n.obs <- table(mesa.model$obs$ID)
> ID1 <- names( n.obs[n.obs>270] )
> ID2 <- names( n.obs[n.obs<=270] )
> Ind.cv1 <- createCV(mesa.model, groups=10,
subset=ID1, Icv.vector=FALSE)
> Ind.cv2 <- createCV(mesa.model, groups=10,
subset=ID2, Icv.vector=FALSE)
Study the number of observations in each group for the two CV-grouping,
> colSums(Ind.cv1)
[1] 275 279 278 277 277 280 277 279 278 278
> colSums(Ind.cv2)
[1] 111 242 215 165 348 256 208 94 80 80
combine to one grouping,
> Ind.cv.final <- Ind.cv1 | Ind.cv2
> colSums(Ind.cv.final)
[1] 386 521 493 442 625 536 485 373 358 358
and compare with the previous grouping.
> table(Ind.cv)
32
5. Cross-validation
Ind.cv
1 2 3 4 5 6 7 8 9 10
438 389 811 556 546 165 228 487 160 797
> ##easier if we sort by number of observations in each group
> rbind(sort(table(Ind.cv)), sort(colSums(Ind.cv.final)))
9 6 7 2 1 8 5 4 10 3
[1,] 160 165 228 389 438 487 546 556 797 811
[2,] 358 358 373 386 442 485 493 521 536 625
If we instead look at which locations that will be excluded from which CVgroup:
> ID.cv <- sapply(split(mesa.model$obs$ID, Ind.cv),unique)
> print(ID.cv)
$`1`
[1] "60370002" "L002" "LC001"
$`2`
[1] "60371002" "60370030" "LC003"
$`3`
[1] "60370016" "60371301" "60374002"
$`4`
[1] "60371201" "60372005"
$`5`
[1] "60591003" "60595001"
$`6`
[1] "60370031" "60375005"
$`7`
[1] "60375001" "60590001"
$`8`
[1] "60370113" "60590007"
$`9`
33
Tutorial for SpatioTemporal
[1] "LC002" "60371602"
$`10`
[1] "60371103" "60371601" "60371701" "L001"
We see that the groups are a lot more even. The four locations in the 10th
group is due to the fact that 60371103 and L001 are colocated.
> mesa.model$D.beta[ID.cv[[10]],ID.cv[[10]]]
60371103 60371601 60371701 L001
60371103 0.00000000 16.36527 43.75141 0.08363892
60371601 16.36527030 0.00000 29.06447 16.44669372
60371701 43.75141252 29.06447 0.00000 43.83429713
L001 0.08363892 16.44669 43.83430 0.00000000
By studying the distance between the locations in the 10th group we see
that the 60371103 and L001 are only 0.084 km apart, which is less than the
min.dist=.1 specified in the createCV-call above. This causes createCV
to lump the two locations together, treating them as “one” location when
creating the CV-grouping.
Instead of creating a list with which location(s) get dropped in each CVgroup
is might be more useful with a vector that, for each location, indicates
which CV-group it belongs to.
> I.col <- apply(sapply(ID.cv,
function(x) mesa.model$locations$ID
%in% x), 1,
function(x) if(sum(x)==1) which(x) else 0)
> names(I.col) <- mesa.model$locations$ID
> print(I.col)
60370002 60370016 60370030 60370031 60370113 60371002
1 3 2 6 8 2
60371103 60371201 60371301 60371601 60371602 60371701
10 4 3 10 9 10
60372005 60374002 60375001 60375005 60590001 60590007
4 3 7 6 7 8
60591003 60595001 L001 L002 LC001 LC002
5 5 10 1 1 9
LC003
2
34
5. Cross-validation
Using this vector we can plot the locations on a map, colour-coded by which
CV-group they belong to, see Figure 5.
> par(mfrow=c(1,1))
> plot(mesa.model$locations$long,
mesa.model$locations$lat,
pch=23+floor(I.col/max(I.col)+.5), bg=I.col,
xlab="Longitude", ylab="Latitude")
> map("county", "california", col="#FFFF0055",
fill=TRUE, add=TRUE)
−118.4 −118.2 −118.0 −117.8
33.7 33.8 33.9 34.0 34.1 34.2
Longitude
Latitude
Figure 5: Location of monitors in the Los Angeles area. The different crossvalidation
groups are indicated by colour and shape of the points, i.e. all
points of the same colour and shape belong to the same cross-validation
group.
35
Tutorial for SpatioTemporal
Having created the CV-grouping we need to estimate the parameters for each
of the CV-groups and then predict the left out observations given the estimated
parameters. The estimation and prediction is described in subsection 5.1
and 5.2 below.
5.1 Cross-Validated Estimation
Parameter estimation for each of the CV-groups is done by the estimateCV
function, which calls estimate.STmodel. The inputs to estimateCV are
similar to those of estimate.STmodel. As described in subsection 4.1 the
estimation function require at least a STmodel and a matrix (or vector) of initial
values, in addition to these estimateCV also requires a vector (or matrix)
describing the CV-grouping, e.g. Ind.cv. Since the parameter estimation for
10 CV-groups using 2 initial values takes some considerable time the results
have been pre-computed.
WARNING: The following steps are time-consuming.
> x.init <- coef(est.mesa.model, pars="cov")[,c("par","init")]
> est.cv.mesa <- estimateCV(mesa.model, x.init, Ind.cv)
ALTERNATIVE: Load pre-computed results.
> data(est.cv.mesa, package="SpatioTemporal")
End of alternative
We first study the results of the estimation.
> print(est.cv.mesa)
Cross-validation parameter estimation for STmodel
with 10 CV-groups and 2 starting points.
Results: 10 converged, 0 not converged.
No fixed parameters.
Estimated function values and convergence info:
value convergence conv eigen.min eigen.all.min
1 5185.641 TRUE TRUE 0.05494492 NA
2 5175.569 TRUE TRUE 1.50038651 NA
3 4699.504 TRUE TRUE 0.23374084 NA
36
5.1 Cross-Validated Estimation
4 4939.731 TRUE TRUE 1.02781382 NA
5 5002.364 TRUE TRUE 1.65927388 NA
6 5748.083 TRUE TRUE 0.12411344 NA
7 5423.788 TRUE TRUE 1.35862268 NA
8 5182.118 TRUE TRUE 0.93230857 NA
9 5513.730 TRUE TRUE 1.34766589 NA
10 4488.807 TRUE TRUE 1.08730352 NA
Here we see that the parameter estimates for all 10 groups have converged, it
also gives the optimal log-likelihood value for each estimate along with convergence
information and the smallest eigenvalue of the Hessian; very small
eigenvalues would indicate that some parameters in that CV-group have large
uncertainties. This information can also be obtained from est.cv.mesa$status.
The estimated parameters for each CV-group can be found in
> head( coef(est.cv.mesa) )
[,1] [,2]
gamma.lax.conc.1500 0.001300551 -0.0007469093
alpha.const.(Intercept) 3.785148274 4.0184242867
alpha.const.log10.m.to.a1 -0.220741894 -0.2861358490
alpha.const.s2000.pop.div.10000 0.033861734 0.0407000867
alpha.const.km.to.coast 0.041069423 0.0340540672
alpha.V1.(Intercept) -0.715712019 -0.7523194640
[,3] [,4]
gamma.lax.conc.1500 0.003210104 0.0008822506
alpha.const.(Intercept) 3.689188133 3.6745268844
alpha.const.log10.m.to.a1 -0.171915684 -0.1806221585
alpha.const.s2000.pop.div.10000 0.024549744 0.0387008796
alpha.const.km.to.coast 0.040649511 0.0405245737
alpha.V1.(Intercept) -0.773503934 -0.7334151753
[,5] [,6]
gamma.lax.conc.1500 0.001056902 0.001144488
alpha.const.(Intercept) 3.865320581 3.672403194
alpha.const.log10.m.to.a1 -0.211921468 -0.209735008
alpha.const.s2000.pop.div.10000 0.037492322 0.037409895
alpha.const.km.to.coast 0.032909451 0.045984052
alpha.V1.(Intercept) -0.684990007 -0.787318698
[,7] [,8]
gamma.lax.conc.1500 -0.003493563 0.001380976
alpha.const.(Intercept) 3.856148801 3.743798417
37
Tutorial for SpatioTemporal
alpha.const.log10.m.to.a1 -0.237413477 -0.210574809
alpha.const.s2000.pop.div.10000 0.038503677 0.047651999
alpha.const.km.to.coast 0.038217637 0.037295311
alpha.V1.(Intercept) -0.751122498 -0.756181663
[,9] [,10]
gamma.lax.conc.1500 0.001416772 0.002142981
alpha.const.(Intercept) 3.701278756 3.511170174
alpha.const.log10.m.to.a1 -0.191588765 -0.146673738
alpha.const.s2000.pop.div.10000 0.042629229 0.041749214
alpha.const.km.to.coast 0.037463438 0.033134254
alpha.V1.(Intercept) -0.765400789 -0.737418155
with uncertainties stored in est.cv.mesa$par.cov.sd and
est.cv.mesa$par.all.sd.
If the option verbose.res=TRUE is used in the call to estimateCV the results
of each estimate.STmodel call are stored in the est.cv.mesa$res.all.
5.2 Cross-Validated Prediction
Once parameters have been estimated predictCV() computes predictions
for each of the CV-groups. This is done by computing the conditional expectations
of the left-out observations, as indicated in by the columns in
Ind.cv, given all other observations and the estimated parameters. Details
can be found in (Lindstr¨om et al., 2013, Szpiro et al., 2010) or Section 4.6 of
vignette("ST_intro", package="SpatioTemporal")
We now compute cross-validation predictions Gaussian, along with temporal
averages based on only the observed time-points.
WARNING: The following steps are time-consuming.
> pred.cv.mesa <- predictCV(mesa.model, est.cv.mesa, LTA=TRUE)
ALTERNATIVE: Load pre-computed results.
> data(pred.cv.mesa, package="SpatioTemporal")
End of alternative
First we examine the results of the predictions:
> print(pred.cv.mesa)
38
5.2 Cross-Validated Prediction
Cross-validation prediction for STmodel with 10 CV-groups.
Predictions for 25 locations and 280 time points.
Temporal averages predicted for 25 locations.
Variances have been computed.
Regression parameters are assumed to be known,
prediction variances do NOT include
uncertainties in regression parameters.
> names(pred.cv.mesa)
[1] "opts" "Ind.cv" "pred.obs" "pred.LTA" "pred.all"
Here the pred.obs contains a data frame with observations, predictions,
prediction variances, and residuals for each observed space-time location
> str(pred.cv.mesa$pred.obs)
'data.frame': 4577 obs. of 10 variables:
$ obs : num 4.58 4.13 4.73 5.35 5.28 ...
$ date : Date, format: "1999-01-13" ...
$ ID : chr "60370002" "60370016" "60370113" "60371002" ...
$ EX.mu : num 4.85 4.77 4.89 4.95 4.79 ...
$ EX.mu.beta: num 4.24 4.44 4.71 4.95 5.06 ...
$ EX : num 4.38 4.51 4.87 5.11 5.21 ...
$ VX : num 0.1148 0.1165 0.0823 0.154 0.0679 ...
$ VX.pred : num 0.1279 0.1293 0.0922 0.1676 0.0803 ...
$ res : num 0.1983 -0.3768 -0.1396 0.2395 0.0742 ...
$ res.norm : num 0.554 -1.048 -0.46 0.585 0.262 ...
Only predictions at observed locations are given in pred.obs, and full predictions
for all space-time locations are collected in pred.all
> str(pred.cv.mesa$pred.all,1)
List of 6
$ EX.mu : num [1:280, 1:25] 4.85 4.66 4.47 4.29 4.12 ...
..- attr(*, "dimnames")=List of 2
$ EX.mu.beta: num [1:280, 1:25] 4.24 4.14 4.04 3.94 3.86 ...
..- attr(*, "dimnames")=List of 2
$ EX : num [1:280, 1:25] 4.38 3.98 3.97 4.22 3.71 ...
..- attr(*, "dimnames")=List of 2
$ VX : num [1:280, 1:25] 0.1148 0.0952 0.0804 0.0701 0.0637 ...
39
Tutorial for SpatioTemporal
..- attr(*, "dimnames")=List of 2
$ VX.pred : num [1:280, 1:25] 0.1279 0.1083 0.0936 0.0832 0.0768 ...
..- attr(*, "dimnames")=List of 2
$ beta :List of 3
which contains fields coresponding to those in pred.mesa.model from predict.STmodel
> names(pred.mesa.model)
[1] "opts" "pars" "beta" "EX.mu"
[5] "EX.mu.beta" "EX" "VX" "VX.pred"
[9] "I"
If requested — LTA=TRUE in the call to predictCV — the temporal averages
are collected in
> str(pred.cv.mesa$pred.LTA)
'data.frame': 25 obs. of 9 variables:
$ obs : num 3.7 4.15 3.24 4.32 4.17 ...
$ ID : chr "60370002" "L002" "LC001" "60370030" ...
$ EX.mu : num 3.83 3.89 3.24 4.11 3.93 ...
$ EX.mu.beta: num 3.72 4.04 2.97 4.28 3.94 ...
$ EX : num 3.72 4.02 2.95 4.3 3.94 ...
$ VX : num 0.0519 0.0482 0.0328 0.0264 0.0524 ...
$ VX.pred : num 0.052 0.0485 0.0331 0.0269 0.0524 ...
$ res : num -0.0196 0.1276 0.2834 0.0186 0.2312 ...
$ res.norm : num -0.0859 0.5793 1.557 0.1133 1.0099 ...
Some standard cross-validation statistics can be obtained through:
> summary(pred.cv.mesa)
Cross-validation predictions for STmodel with 10 CV-groups.
Predictions for 4577 observations.
Temporal averages for 25 locations.
RMSE:
EX.mu EX.mu.beta EX
obs 0.4260103 0.3564484 0.3150911
40
5.2 Cross-Validated Prediction
average 0.3115791 0.2248556 0.2246617
R2:
EX.mu EX.mu.beta EX
obs 0.6533988 0.7573483 0.8103896
average 0.3747796 0.6743855 0.6749468
Coverage of 95% prediction intervals:
EX
obs 0.9228752
average 0.9200000
Here summary.predCVSTmodel computes RMSE, R2
, and coverage of prediction
intervalls for the observations and for the long term average at each
location.
If temporal averages were not computed by predictCV, the option LTA=TRUE
can be used in summary.predCVSTmodel to obtain some statistics for the
averages (not coverage since the variance will not be computed). We can
also ask summary to perform a transformation of observations and predictions
before computing the CV-statistics. This transform is not biased corrected,
and for the exponential case the better option is often to use the transform
option of predict and predictCV.
5.2.1 Residual Analysis
Before we start with a more thorough residual analysis, we need to create an
indicator vector for which season each observation belongs to.
> I.season <- as.factor(as.POSIXlt(pred.cv.mesa$pred.obs$date)$mon+1)
> levels(I.season) <- c(rep("Winter",2), rep("Spring",3),
rep("Summer",3), rep("Fall",3), "Winter")
Given this we can investigate how many observations we have during each
season.
> table(I.season)
I.season
Winter Spring Summer Fall
1096 1223 1172 1086
41
Tutorial for SpatioTemporal
We now take a look at the residuals from the prediction, to do this we use the
pred.cv.mesa object computed above which contains prediction residuals.
First we’ll examine a residual QQ-plot to assess the normality of the residuals,
both raw and normalised, shown in Figure 6.
> par(mfrow=c(1,2), mar=c(3,2,1,1), pty="s")
> qqnorm(pred.cv.mesa, col=I.season, line=2)
> qqnorm(pred.cv.mesa, norm=TRUE, main="Normalised residuals",
col=I.season)
> legend("bottomright", legend=as.character(levels(I.season)),
pch=1, col=1:nlevels(I.season))
Here we have used the I.season indicator to colour code our observations,
this should help us to detect any seasonal effects on the predictions.

In [ ]:
Figure 8: Residual scatter plots, by location type.
45
Tutorial for SpatioTemporal
Acknowledgements
Data used in the examples has been provided by the Multi-Ethnic Study
of Atherosclerosis and Air Pollution (MESA Air). Details regarding
the data can be found in Cohen et al. (2009), Wilton et al. (2010).
Although this tutorial and development of the package described there in has
been funded wholly or in part by the United States Environmental Protection
Agency through assistance agreement CR-834077101-0 and grant
RD831697 to the University of Washington, it has not been subjected to
the Agency’s required peer and policy review and therefore does not necessarily
reflect the views of the Agency and no official endorsement should be
inferred.
Travel for Johan Lindstr¨om has been paid by STINT Grant IG2005-2047.
Additional funding was provided by grants to the University of Washington
from the National Institute of Environmental Health Sciences (P50
ES015915) and the Health Effects Institute (4749-RFA05-1A/06-10).
46
REFERENCES
References
Cohen, M. A., Adar, S. D., Allen, R. W., Avol, E., Curl, C. L., Gould,
T., Hardie, D., Ho, A., Kinney, P., Larson, T. V., Sampson, P. D., Sheppard,
L., Stukovsky, K. D., Swan, S. S., Liu, L.-J. S., and Kaufman, J. D.
(2009). Approach to estimating participant pollutant exposures in the
Multi-Ethnic Study of Atherosclerosis and air pollution (MESA air). Environmental
Science & Technology, 43(13):4687–4693.
EPA (1992). User’s guide to CAL3QHC version 2.0: A modeling methodology
for predicting pollutant concentrations near roadway intersections. Technical
Report EPA-454/R-92-006, U.S. Environmental Protection Agency,
Research Triangle Park, NC, USA.
Hastings, W. (1970). Monte Carlo sampling methods using Markov chains
and their applications. Biometrika, 57:97–109.
Lindstr¨om, J., Szpiro, A. A., Sampson, P. D., Oron, A., Richards, M., Larson,
T., and Sheppard, L. (2013). A flexible spatio-temporal model for air
pollution with spatial and spatio-temporal covariates. Under revision for
Environmental and Ecological Statistics, TBD:?–?
Lindstr¨om, J., Szpiro, A. A., Sampson, P. D., Sheppard, L., Oron, A.,
Richards, M., and Larson, T. (2011). A flexible spatio-temporal model
for air pollution: Allowing for spatio-temporal covariates. Technical Report
Working Paper 370, UW Biostatistics Working Paper Series.
Mercer, L. D., Szpiro, A. A., Sheppard, L., Lindstr¨om, J., Adar, S. D., Allen,
R. W., Avol, E. L., Oron, A. P., Larson, T., Liu, L.-J. S., and Kaufman,
J. D. (2011). Comparing universal kriging and land-use regression
for predicting concentrations of gaseous oxides of nitrogen (NOx) for the
multi-ethnic study of atherosclerosis and air pollution (MESA air). Atmo.
Environ., 45(26):4412–4420.
MESA Air Data Team (2010). Documentation of MESA air implementation
of the Caline3QHCR model. Technical report, University of Washington,
Seattle, WA, USA.
Metropolis, N., Rosenbluth, A., Rosenbluth, M., Teller, A., and Teller, E.
(1953). Equations of state calculations by fast computing machines. J.
Chem. Phys., 21:1087–1092.
47
Tutorial for SpatioTemporal
Roberts, G., Gelman, A., and Gilks, W. (1997). Weak convergence and
optimal scaling of random walk metropolis algorithms. Ann. Appl. Probab.,
7:110–120.
Sampson, P. D., Szpiro, A. A., Sheppard, L., Lindstr¨om, J., and Kaufman,
J. D. (2011). Pragmatic estimation of a spatio-temporal air quality model
with irregular monitoring data. Atmo. Environ., 45(36):6593–6606.
Szpiro, A. A., Sampson, P. D., Sheppard, L., Lumley, T., Adar, S., and
Kaufman, J. (2010). Predicting intra-urban variation in air pollution concentrations
with complex spatio-temporal dependencies. Environmetrics,
21(6):606–631.
Wilton, D., Szpiro, A. A., Gould, T., and Larson, T. (2010). Improving
spatial concentration estimates for nitrogen oxides using a hybrid meteorological
dispersion/land use regression model in Los Angeles, CA and
Seattle, WA. Sci. Total Environ., 408(5):1120–1130.



